In [ ]:
##Ejercicio 1 

import pandas as pd
import matplotlib.pyplot as plt

## 2) 
#a) Importamos la base de datos, y nos quedamos unicamente con las observaciones pertenecientes a a los aglomerados de Ciudad Autónoma de Buenos Aires o Gran Buenos Aires.
df = pd.read_excel('usu_individual_T423.xlsx')

df = df[df['AGLOMERADO'].isin([32, 33])]




In [ ]:
#b) Eliminamos observaciones con valores atipicos, como edades o ingresos negativos
df = df[df['CH06'] > 0]

df = df[df['P47T'] >= 0]

df = df[df['ITF'] >= 0]

In [ ]:
#c)
# Graficamos la composición por sexo
sexo_composicion = df['CH04'].value_counts().sort_index()
sexo_composicion.plot(kind='bar', color=['pink', 'blue']) 
plt.title('Composición por Sexo')
plt.xlabel('Sexo')
plt.ylabel('Cantidad')
plt.xticks(ticks=[0, 1], labels=['Mujeres', 'Hombres'], rotation=0)

plt.show()

In [ ]:
#d) Correlograma
data_corr = df[['CH04','CH07', 'CH08', 'NIVEL_ED', 'ESTADO', 'CAT_INAC', 'IPCF']]
corr = data_corr.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
)

In [ ]:
#e) Desocupado, inactivos

desocupados = len(df[df['ESTADO']==2])
inactivos = len(df[df['ESTADO']==3])

print('Cantidad de desocupados:', desocupados)
print('Cantidad de inactivos:', inactivos)

#Eliminamos todas las variables que no tengan Estado de desocupado o inactivo
des_in = df[(df['ESTADO'] == 1) | (df['ESTADO']==2) | (df['ESTADO'] == 3)]

estado_ipcf = des_in.groupby('ESTADO')['IPCF'].mean().reset_index()

In [ ]:
#f) Necesidades Energeticas

df_adulto = pd.read_excel("C:/Users/tomas/Documents/UdeSA/Cuarto Año/Primer Cuatri/Big Data/TP3/tabla_adulto_equiv.xlsx")

#Creamos dummies por cada combinacion grupo/genero

bins = [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 29, 45, 60, 75, float('inf')]
labels = [
    'less_than_1', '1_year', '2_year', '3_year', '4_year', '5_year', '6_year', '7_year', '8_year', '9_year', '10_year', 
    '11_year', '12_year', '13_year', '14_year', '15_year', '16_year', '17_year', '18_to_29', '30_to_45', 
    '46_to_60', '61_to_75', 'more_than_75'
]

df['age_group'] = pd.cut(df['CH06'], bins=bins, labels=labels)

#Dummie por genero y edad
for sex in [1, 2]:
    for label in labels:
        dummy_name = f'sex_{sex}_age_{label}'
        df[dummy_name] = ((df['CH04'] == sex) & (df['age_group'] == label)).astype(int)


In [ ]:
#Mapeo de necesidades energeticas segun edad y genero
energy_needs = {
    'less_than_1': {1: 0.35, 2: 0.35},
    '1_year': {1: 0.37, 2: 0.37},
    '2_year': {1: 0.46, 2: 0.46},
    '3_year': {1: 0.51, 2: 0.51},
    '4_year': {1: 0.55, 2: 0.55},
    '5_year': {1: 0.60, 2: 0.60},
    '6_year': {1: 0.64, 2: 0.64},
    '7_year': {1: 0.66, 2: 0.66},
    '8_year': {1: 0.68, 2: 0.68},
    '9_year': {1: 0.69, 2: 0.69},
    '10_year': {1: 0.70, 2: 0.79},
    '11_year': {1: 0.72, 2: 0.82},
    '12_year': {1: 0.74, 2: 0.85},
    '13_year': {1: 0.76, 2: 0.90},
    '14_year': {1: 0.76, 2: 0.96},
    '15_year': {1: 0.77, 2: 1.00},
    '16_year': {1: 0.77, 2: 1.03},
    '17_year': {1: 0.77, 2: 1.04},
    '18_to_29': {1: 0.76, 2: 1.02},
    '30_to_45': {1: 0.77, 2: 1.00},
    '46_to_60': {1: 0.76, 2: 1.00},
    '61_to_75': {1: 0.67, 2: 0.83},
    'more_than_75': {1: 0.63, 2: 0.74}
}

In [ ]:
#Cambiar valor 1 de dummie por valor energetico correspondiente a esa categoria
for sex in [1, 2]:
    for label in labels:
        dummy_name = f'sex_{sex}_age_{label}'
        energy_value = energy_needs[label][sex]
        df[dummy_name] = df[dummy_name] * energy_value


In [ ]:
#Nos quedamos con los valores de necesidad energetica en una nueva variable y eliminamos las dummies
def get_positive_energy_value(row):
    for sex in [1, 2]:
        for label in labels:
            dummy_name = f'sex_{sex}_age_{label}'
            if row[dummy_name] > 0:
                return row[dummy_name]
    return 0

df['necesidades_energeticas'] = df.apply(get_positive_energy_value, axis=1)

df = df.loc[:, ~df.columns.str.startswith('sex_')]

df.drop(columns=['age_group'], inplace=True)